In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [2]:
test_trans = pd.read_csv('./data/transactions_test.csv', parse_dates=["TX_TS"])
train_trans =  pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])
merchants_full = pd.read_csv('./data/merchants.csv')

terminals_full = pd.read_csv('./data/terminals.csv')

customers_full = pd.read_csv('./data/customers.csv')

C:\Users\joshu\AppData\Local\Temp\ipykernel_31264\4058358162.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  train_trans =  pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])


In [3]:
test_trans = pd.merge(test_trans,terminals_full,on='TERMINAL_ID')
test_trans = pd.merge(test_trans,customers_full,on='CUSTOMER_ID')
test_trans = pd.merge(test_trans,merchants_full,on='MERCHANT_ID')


In [4]:
train_trans = pd.merge(train_trans,terminals_full,on='TERMINAL_ID')
train_trans = pd.merge(train_trans,customers_full,on='CUSTOMER_ID')
train_trans = pd.merge(train_trans,merchants_full,on='MERCHANT_ID')


In [5]:
test_trans['is_test'] = 1

In [6]:
train_trans['is_test'] = 0

In [7]:
all_trans = pd.concat([train_trans,test_trans])

In [8]:
all_trans.columns

Index(['TX_ID', 'TX_TS', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'TX_FRAUD',
       'TRANSACTION_GOODS_AND_SERVICES_AMOUNT', 'TRANSACTION_CASHBACK_AMOUNT',
       'CARD_EXPIRY_DATE', 'CARD_DATA', 'CARD_BRAND', 'TRANSACTION_TYPE',
       'TRANSACTION_STATUS', 'FAILURE_CODE', 'FAILURE_REASON',
       'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE', 'MERCHANT_ID',
       'IS_RECURRING_TRANSACTION', 'ACQUIRER_ID', 'CARDHOLDER_AUTH_METHOD',
       'x_terminal_id', 'y_terminal__id', 'x_customer_id', 'y_customer_id',
       'BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE',
       'TAX_EXCEMPT_INDICATOR', 'OUTLET_TYPE', 'ACTIVE_FROM', 'TRADING_FROM',
       'ANNUAL_TURNOVER_CARD', 'ANNUAL_TURNOVER', 'AVERAGE_TICKET_SALE_AMOUNT',
       'PAYMENT_PERCENTAGE_FACE_TO_FACE', 'PAYMENT_PERCENTAGE_ECOM',
       'PAYMENT_PERCENTAGE_MOTO', 'DEPOSIT_REQUIRED_PERCENTAGE',
       'DEPOSIT_PERCENTAGE', 'DELIVERY_SAME_DAYS_PERCENTAGE',
       'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERC

In [9]:
useless_columns = ['MERCHANT_ID','CARD_EXPIRY_DATE','CARD_DATA','LEGAL_NAME','FOUNDATION_DATE','ACTIVE_FROM','TRADING_FROM']
all_trans.drop(useless_columns,axis=1,inplace=True)

In [10]:
all_trans.columns

Index(['TX_ID', 'TX_TS', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'TX_FRAUD',
       'TRANSACTION_GOODS_AND_SERVICES_AMOUNT', 'TRANSACTION_CASHBACK_AMOUNT',
       'CARD_BRAND', 'TRANSACTION_TYPE', 'TRANSACTION_STATUS', 'FAILURE_CODE',
       'FAILURE_REASON', 'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE',
       'IS_RECURRING_TRANSACTION', 'ACQUIRER_ID', 'CARDHOLDER_AUTH_METHOD',
       'x_terminal_id', 'y_terminal__id', 'x_customer_id', 'y_customer_id',
       'BUSINESS_TYPE', 'MCC_CODE', 'TAX_EXCEMPT_INDICATOR', 'OUTLET_TYPE',
       'ANNUAL_TURNOVER_CARD', 'ANNUAL_TURNOVER', 'AVERAGE_TICKET_SALE_AMOUNT',
       'PAYMENT_PERCENTAGE_FACE_TO_FACE', 'PAYMENT_PERCENTAGE_ECOM',
       'PAYMENT_PERCENTAGE_MOTO', 'DEPOSIT_REQUIRED_PERCENTAGE',
       'DEPOSIT_PERCENTAGE', 'DELIVERY_SAME_DAYS_PERCENTAGE',
       'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE',
       'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE', 'is_test', 'ID_JOIN'],
      dtype='object')

In [11]:
all_trans.loc[all_trans['FAILURE_CODE'].isna(),['FAILURE_CODE']] = 99

In [12]:
all_trans.loc[all_trans['FAILURE_REASON'].isna(),['FAILURE_REASON']] = 'Non Failure'

In [13]:
all_trans.dropna(subset=['ACQUIRER_ID','CARDHOLDER_AUTH_METHOD'], inplace=True)

In [14]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)
all_trans['TX_DURING_NIGHT']=all_trans.TX_TS.apply(is_night)


In [15]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)
all_trans['TX_DURING_WEEKEND']=all_trans.TX_TS.apply(is_weekend)


In [16]:
all_trans.drop(['TX_TS'],axis=1,inplace=True)

In [17]:
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == 'True',['IS_RECURRING_TRANSACTION']] = 1
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == True,['IS_RECURRING_TRANSACTION']] = 1
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == 'False',['IS_RECURRING_TRANSACTION']] = 0
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == 'Fals',['IS_RECURRING_TRANSACTION']] = 0
all_trans.loc[all_trans['IS_RECURRING_TRANSACTION'] == False,['IS_RECURRING_TRANSACTION']] = 0

all_trans['IS_RECURRING_TRANSACTION'] = all_trans['IS_RECURRING_TRANSACTION'].apply(pd.to_numeric)

In [18]:
one_hot_encoded_col = ['CARD_BRAND','TRANSACTION_TYPE','TRANSACTION_STATUS','FAILURE_CODE','FAILURE_REASON','TRANSACTION_CURRENCY','CARD_COUNTRY_CODE','ACQUIRER_ID','CARDHOLDER_AUTH_METHOD','BUSINESS_TYPE','OUTLET_TYPE','TX_DURING_NIGHT']
all_trans = pd.get_dummies(all_trans, columns = one_hot_encoded_col,)

In [19]:
np.where(all_trans.dtypes == 'object' )

(array([0], dtype=int64),)

In [20]:
all_trans.select_dtypes(include=['object']).columns

Index(['TX_ID'], dtype='object')

In [21]:
test_df = all_trans.loc[all_trans['is_test'] == 1].drop(columns=['is_test','TX_FRAUD'])
train_df = all_trans.loc[all_trans['is_test'] == 0].drop(columns=['TX_ID','is_test'])

In [22]:
location = 'feature_set_4'
test_df.to_csv(F'./data/{location}/test_features.csv',index=False)

In [23]:
train_df.to_csv(F'./data/{location}/train_features.csv',index=False)

y = train_df['TX_FRAUD']
X = train_df.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

X_train_full.to_csv(F'./data/{location}/X_train_full.csv',index=False)
X_valid_full.to_csv(F'./data/{location}/X_valid_full.csv',index=False)
y_train.to_csv(F'./data/{location}/y_train.csv',index=False)
y_valid.to_csv(F'./data/{location}/y_valid.csv',index=False)

In [24]:
other_cols = list(set(all_trans.columns)-set(['TX_ID']))
other_cols

['TRANSACTION_CURRENCY_CAD',
 'BUSINESS_TYPE_Corporations',
 'CARD_COUNTRY_CODE_DE',
 'CARD_BRAND_Discover',
 'FAILURE_REASON_Invalid amount',
 'TRANSACTION_CURRENCY_GBP',
 'FAILURE_REASON_Unable to go online; offline-declined',
 'OUTLET_TYPE_Face to Face',
 'CARD_BRAND_MasterCard',
 'FAILURE_REASON_Credit issuer unavailable',
 'TX_DURING_NIGHT_0',
 'FAILURE_REASON_Invalid transaction',
 'FAILURE_CODE_99',
 'TRANSACTION_STATUS_Authorized',
 'ACQUIRER_ID_ACQ6',
 'TRANSACTION_TYPE_Cash\xa0Advance/Withdrawal',
 'CARD_COUNTRY_CODE_SA',
 'DELIVERY_SAME_DAYS_PERCENTAGE',
 'FAILURE_REASON_Insufficient funds/over credit limit / Not sufficient funds',
 'CARDHOLDER_AUTH_METHOD_Offline enciphered PIN',
 'FAILURE_REASON_Do not honor',
 'PAYMENT_PERCENTAGE_ECOM',
 'FAILURE_REASON_Restricted card',
 'FAILURE_CODE_6',
 'TRANSACTION_TYPE_Purchase',
 'BUSINESS_TYPE_S Corporations',
 'FAILURE_CODE_59',
 'TRANSACTION_STATUS_Rejected',
 'CARD_COUNTRY_CODE_US',
 'FAILURE_REASON_Non Failure',
 'CARDHOLDER_A

In [25]:
scaler = MinMaxScaler()
scaler.fit(all_trans[other_cols])

MinMaxScaler()

In [26]:
all_trans_normalised = pd.DataFrame(scaler.transform(all_trans[other_cols]), index=all_trans[other_cols].index, columns=other_cols)

In [27]:
all_trans_normalised['TX_ID'] = all_trans['TX_ID']

In [28]:
test_df = all_trans_normalised.loc[all_trans['is_test'] == 1].drop(columns=['is_test','TX_FRAUD'])
train_df = all_trans_normalised.loc[all_trans['is_test'] == 0].drop(columns=['TX_ID','is_test'])

In [29]:
location = 'feature_set_4_normalised'
test_df.to_csv(F'./data/{location}/test_features.csv',index=False)

In [30]:
train_df.to_csv(F'./data/{location}/train_features.csv',index=False)

y = train_df['TX_FRAUD']
X = train_df.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

X_train_full.to_csv(F'./data/{location}/X_train_full.csv',index=False)
X_valid_full.to_csv(F'./data/{location}/X_valid_full.csv',index=False)
y_train.to_csv(F'./data/{location}/y_train.csv',index=False)
y_valid.to_csv(F'./data/{location}/y_valid.csv',index=False)

In [31]:
all_trans.isna().sum()

TX_ID                                         0
CUSTOMER_ID                                   0
TERMINAL_ID                                   0
TX_AMOUNT                                     0
TX_FRAUD                                  71138
                                          ...  
OUTLET_TYPE_Ecommerce                         0
OUTLET_TYPE_Face to Face                      0
OUTLET_TYPE_Face to Face and Ecommerce        0
TX_DURING_NIGHT_0                             0
TX_DURING_NIGHT_1                             0
Length: 126, dtype: int64

In [62]:
og_test = pd.read_csv('./data/transactions_test.csv', parse_dates=["TX_TS"])

In [66]:
set(og_test['TX_ID'])- set(test_df['TX_ID'])

{'ddaa070acea087eae360225e92c1609cea905e43'}